# CCP

> flashing interface for CCP
> For now, only support CCP on CAN via VCAN or Kvaser CAN interface

In [ ]:
#| default_exp ccp.virtual

In [ ]:
#| hide
from nbdev.showdoc import show_doc
from fastcore.test import *

In [ ]:
#| export
import os
import sys
import git
import argparse
from InquirerPy import inquirer
from InquirerPy.validator import EmptyInputValidator
from InquirerPy.base.control import Choice
from pydantic import BaseModel, Field,  ConfigDict, model_validator, conlist, conint, computed_field, confloat, ValidationError
from pydantic.functional_validators import AfterValidator
from typing import Optional, TypedDict, Union, Literal
from typing_extensions import Annotated
from enum import StrEnum
from pprint import pprint, PrettyPrinter

In [ ]:
#| export
import subprocess
from multiprocessing import Manager
from multiprocessing.managers import DictProxy
import cantools
from cantools.database import Message as MessagerTpl
from cantools.database.can.database import Database
import contextlib

In [ ]:
#| export
import pandas as pd
import numpy as np
import struct

In [ ]:
# #|export
# from candycan.data_link_socketcan import done, send_msg

In [ ]:
#| export 
from candycan.a2l import (
    list_of_strings,
    XCPCalib,
    XCPData,
    XCPConfig,
    Get_XCPCalib_From_XCPJSon,
    Generate_Init_XCPData_From_A2L,
)


In [ ]:
#| export
from scapy.all import (
    raw, rdpcap, wrpcap, load_contrib, hexdump,
    ls, conf, load_layer, IP, Ether, TCP
)
# Ether, TCP, hexdump, raw, rdpcap, load_contrib, conf, load_layer, 
# CANSocket, CAN, wrpcap, CCP, CRO, CONNECT, GET_SEED, UNLOCK, GET_DAQ_SIZE

In [ ]:
#| hide
# running on a physical linux machine (blue pill), otherwise red pill
blue_pill = False if os.getenv("GITHUB_ACTIONS") else True

In [ ]:
#| export
load_layer("can")  # CAN
conf.contribs['CANSocket'] = {'use-python-can': False}
load_contrib("cansocket") # CANSocket
load_contrib("automotive.ccp")  # CCP, CRO, CONNECT, DISCONNECT, GET_SEED, UNLOCK, GET_DAQ_SIZE

In [ ]:
#| export
pp = PrettyPrinter(indent=4, width=80, compact=True)

In [ ]:
#| export 
repo = git.Repo("./", search_parent_directories=True)  # get the Repo object of tspace
if os.path.basename(repo.working_dir) != "candycan":  # I'm in the parent repo!
    repo = repo.submodule("candycan").module()
pprint(repo.working_dir)

In [ ]:
#| export
def get_argparser() -> argparse.ArgumentParser:
	"""Summary
	Get argument parser for command line arguments

	Returns:
		argparse.ArgumentParser: _description_
	"""
	parser = argparse.ArgumentParser(description='XCP Processing')

	parser.add_argument(
		'--protocol',
		type=str,
		choices=['CCP', 'XCP'],
		default='CCP',
		help='Protocol to use: CCP/XCP',
	)

	parser.add_argument(
		'--can_type',
		type=str,
		choices=['NATIVE', 'PYTHON'],
		default='NATIVE',
		help='CAN type to use: NATIVE/PYTHON',
	)

	parser.add_argument(
		'--bus_type',
		type=str,
		choices=['SOCKET', 'VIRTUAL', 'KVASER'],
		default='VIRTUAL',
		help='Bus type to use: SOCKET/VIRTUAL/KVASER',
	)

	parser.add_argument(
		'--download',
		default=False,
		help='Download or upload: default is download(host->target)',
		action='store_true',
	)

	parser.add_argument(
		'--diff_mode',
		default=True,
		help='use differential mode for flashing',
		action='store_true',
	)

	parser.add_argument(
		'--diff_threshold',
		type=float,
		default=0.001,
		help='Threshold as different float value for differential mode',
	)

	parser.add_argument(
		'--bit_rate',
		type=int,
		choices=range(1_000_000),
		default=500_000,
		help='Bit rate for CAN bus, maximal 1Mbps',
	)

	parser.add_argument(
		'--time_out',
		type=float,
		default=1.0,
		help='Time out for CAN bus response',
	)

	parser.add_argument(
		"--station_address",
		type=int,
		default=0,
		help='Station address of ECU for CCP protocol',
	)

	parser.add_argument(
		"--download_can_id",
		type=int,
		default=630,
		help='Download CAN message ID for CCP protocol',
	)

	parser.add_argument(
		"--upload_can_id",
		type=int,
		default=631,
		help='Upload CAN message ID for CCP protocol',
	)

	parser.add_argument(
		'--a2l', 
		type=str,
        default=repo.working_dir+'/res/VBU_AI.json',
		help='a2l json file path')

	parser.add_argument(
		"--node-path",
		type=str,
		default=r"/PROJECT/MODULE[]",
		help="node path to search for calibration parameters",
	)

	parser.add_argument(
		"--leaves",
		type=list_of_strings,
		default=r"TQD_trqTrqSetNormal_MAP_v, " 
				r"VBU_L045A_CWP_05_09T_AImode_CM_single, " 
				r"Lookup2D_FLOAT32_IEEE, " 
				r"Lookup2D_X_FLOAT32_IEEE, " 
				r"Scalar_FLOAT32_IEEE, " 
				r"TQD_vVehSpd, "
				r"TQD_vSgndSpd_MAP_y, "
				r"TQD_pctAccPedPosFlt, "
				r"TQD_pctAccPdl_MAP_x",
			help="leaf nodes to search for",
	)

	parser.add_argument(
		'--channel_serial_number', 
		type=int,
		default=3,
		help='CAN channel serial number for flashing')

	parser.add_argument(
		'--input', 
		type=str,
        default=repo.working_dir+'/res/download.json',
		help='Input file path')

	parser.add_argument(
		'--output', 
		type=str, 
        default=repo.working_dir+'/res/output.json',
		help='Output file path')
	return parser

In [ ]:
show_doc(get_argparser)

In [ ]:
parser = get_argparser()
args = parser.parse_args(
    args=[
        '--protocol', 'CCP',
        '--can_type', 'NATIVE',
        '--bus_type', 'VIRTUAL',
        '--download',
        '--diff_mode',
        '--diff_threshold', '0.001',
        '--bit_rate', '500_000',
        '--time_out', '1.0',
        '--station_address', 0,
        '--download_can_id', '630',
        '--upload_can_id', '631',
        '--a2l', repo.working_dir+'/res/VBU_AI.json',
        '--node-path', r'/PROJECT/MODULE[]',
        '--leaves', r'TQD_trqTrqSetNormal_MAP_v, VBU_L045A_CWP_05_09T_AImode_CM_single, Lookup2D_FLOAT32_IEEE, Lookup2D_X_FLOAT32_IEEE, Scalar_FLOAT32_IEEE, TQD_vVehSpd, TQD_vSgndSpd_MAP_y, TQD_pctAccPedPosFlt, TQD_pctAccPdl_MAP_x',
        '--channel_serial_number', '3',
        '--input', repo.working_dir+'/res/download.json',
        '--output', repo.working_dir+'/res/output.json',
    ]
)

In [ ]:
args.download_can_id, args.channel_serial_number, args.upload_can_id
args.diff_threshold, hex(args.station_address), args.time_out
args.__dict__

## Types definition

In [ ]:
#| export
CAN_TYPES = set(['NATIVE','PYTHON'])  # Navtive: Native CAN: PYTHON: Python CAN
# class CanType(StrEnum):
#     NATIVE = "NATIVE"
#     PYTHON = "PYTHON"
CAN_TYPES

In [ ]:
#| export
def check_can_type(c: str) -> str:
    """Summary
    Check if the CAN type is valid

    Args:
        can_type (str): CAN type to be checked

    Returns:
        str: CAN type if valid

    Raises:
        ValueError: if CAN type is invalid
    """
    if c.upper() not in CAN_TYPES:
        raise ValueError(f"Invalid CAN type: {c}, valid types are: {CAN_TYPES}")
    return c

CANType = Annotated[str, AfterValidator(check_can_type)]

In [ ]:
# # CanType
# native_can_type = CanType('NATIVE')
# native_can_type.lower()
# isinstance(native_can_type, CanType)
# isinstance('NATIVE', CanType)

In [ ]:
#| export
BUS_TYPES = set(['SOCKET', 'VIRTUAL', 'KVASER', 'PCANUSB', 'IXXAT', 'VECTOR', 'SERIAL', 'NEOVI'])
# class BusType(StrEnum):
#     SOCKET = "SOCKET"
#     VIRTUAL = "VIRTUAL"
#     KVASER = "KVASER"
#     PCANUSB = "PCANUSB"
#     IXXAT = "IXXAT"
#     VECTOR = "VECTOR"
#     SERIAL = "SERIAL"
#     NEOVI = "NEOVI"


In [ ]:
#| export
def check_bus_type(b: str) -> str:
    """Summary
    Check if the CAN bus type is valid

    Args:
        b (str): Python CAN bus type to be checked

    Returns:
        str: Python CAN bus type if valid

    Raises:
        ValueError: if CAN bus type is invalid
    """
    if b.upper() not in BUS_TYPES:
        raise ValueError(f"Invalid Python CAN bus type: {b}, valid types are: {BUS_TYPES}")
    return b

BusType = Annotated[str, AfterValidator(check_bus_type)]

In [ ]:
#|export
class CANFilter(BaseModel):
    """Summary
    CAN filter for Python CAN bus

    Attributes:
        can_id (int): CAN message ID
        can_mask (int): CAN message mask
    """
    can_id: int = Field(default=630,gt=0,title="CAN message ID",description="CAN message ID")
    can_mask: int = Field(default=0x7ff,gt=0,title="CAN message mask",description="CAN message mask")

In [ ]:
#| export
class ScapyCANSpecs(BaseModel):
    can_type: CANType = Field(frozen=True, default='NATIVE', description='CAN type: NATIVE/PYTHON')
    bus_type: BusType = Field(frozen=True, default='VIRTUAL', description='Python CAN bus type')
    channel_serial_number: int = Field(frozen=True, default=3, ge=0, lt=500,description='CAN channel')
    download_can_id: int = Field(default=630, gt=0, description='Download CAN message ID')
    upload_can_id: int = Field(default=630, gt=0, description='Upload CAN message ID')
    can_filters: Optional[list[CANFilter]] = Field(default=None, description='CAN filters')
    bit_rate: int = Field(default=500_000, gt=0, lt=1_000_000, description='CAN bit rate')
    time_out: float = Field(default=1.0, gt=0.0, lt=10.0, description='CAN time out')
    station_address: int = Field(default=0x00, ge=0, lt=0xff, description='CAN station address')
    cntr: int = Field(default=0, ge=0, lt=1_000_000, description='CAN counter')
    receive_own_messages: bool = Field(default=True, description='Receive own messages')
    download_upload: bool = Field(default=True, description='Download if True or upload if False')
    diff_mode: bool = Field(default=False, description='Differential flashing (download)')
    diff_threshold: float = Field(default=1e-3, description='Threshold for float value difference')
    last_download_data: Optional[XCPData] = Field(default=None, description='Last download data')
    
    @computed_field
    def channel(self) -> Union[str, int]:
        """Summary
        Get the CAN channel as str for SocketCAN or virtual CAN, as int for Vector

        Returns:
            str: CAN channel for SocketCAN virtual CAN
            int: for Vector CAN
        """
        match self.bus_type:
            case 'VIRTUAL':
                return 'vcan' + str(self.channel_serial_number)
            case 'SOCKET':
                return 'can' + str(self.channel_serial_number)
            case 'VECTOR' | "KVASER":
                return self.channel_serial_number
            case 'PCAN':
                return 'PCAN_USBBUS' + str(self.channel_serial_number)
            case _:
                raise NotImplementedError(f"Bus type: {self.bus_type} not implemented yet")
                


In [ ]:
try:
    m = ScapyCANSpecs(can_type='NATIVE', bus_type='VIRTUAL')
except ValidationError as exc:
    print(exc)

pprint(m.model_dump())

In [ ]:

def emulate_call(m: ScapyCANSpecs):
    m.cntr += 1

for i in range(3):
    emulate_call(m)
    print(f"{i}: counter {m.cntr}")

In [ ]:
xcp_calib_from_xcpjson = Get_XCPCalib_From_XCPJSon(args.input)
xcp_calib_from_xcpjson

In [ ]:

xcp_data = Generate_Init_XCPData_From_A2L(
    a2l=args.a2l, keys=args.leaves, node_path=args.node_path
)

#  address from xcp data file should align with the address from xcp calib file
test_eq(xcp_data.address, xcp_calib_from_xcpjson.data[0].address)

# validate the model
try:
    XCPData.model_validate(xcp_data)
except ValidationError as exc:
    print(exc)

In [ ]:
# type(args.channel), type(args.download_id), args.upload_id, args.download, args.diff_flashing

In [ ]:
xcp_data.value = xcp_calib_from_xcpjson.data[0].value
pprint(xcp_data)
xcp_data.value_array_view[0,2], xcp_data.value_array_view[2,0]

In [ ]:

xcp_calib = XCPCalib(
    config=XCPConfig(
        channel=args.channel_serial_number, download=args.download_can_id, upload=args.upload_can_id
    ),
    data=[xcp_data],
)

In [ ]:
pprint(xcp_calib)

In [ ]:
npa =  xcp_calib.data[0].value_array_view
npa.shape, npa.dtype, npa
len(xcp_calib.data[0].value_bytes), xcp_calib.data[0].value_bytes

len(npa.tobytes()), npa.tobytes()
test_eq(npa.tobytes(), xcp_calib.data[0].value_bytes)
xcp_calib.data[0].value

In [ ]:
# buffer = [i.hex() for x in npa for i in x]
# # buffer[::-1]
# len(buffer)
# buffer

In [ ]:

# buffer = npa.tobytes()

# pprint(buffer), len(buffer)
# xcp_calib.data[0].value, len(xcp_calib.data[0].value)

In [ ]:
addr = bytes('7000aa2a', 'utf-8')
a = 0x7000aa2a
a

In [ ]:
npb = npa[::-1]
# npb
buffer = [struct.pack("<f", x) for x in np.nditer(npa)]
# buffer
len(buffer)

### investigate int type and type size

In [ ]:
d = xcp_calib.data[0]
add = int(d.address, base=16)
d.address, add
hex(add), type(add), sys.getsizeof(add)
struct.calcsize('h'), struct.calcsize('i'), struct.calcsize('l'), struct.calcsize('L')
type(d.address),len(d.address)

In [ ]:
#| export
def npa_to_packed_buffer(a: np.ndarray) -> str:
    """ convert a numpy array to a packed string buffer for flashing
    TODO: implementation as numpy ufunc

    Args:
        a (np.ndarray): input numpy array for flashing

    Returns:
        str: packed string buffer for flashing
    """
    b = [struct.pack("<f", x).hex() for x in np.nditer(a)]
    return ''.join(b)

In [ ]:

# buffer = [struct.pack("<f", x).hex() for x in np.nditer(npa)]
# buffer[::-1]
# len(buffer)
# buffer
# data = ''.join(buffer)
data = npa_to_packed_buffer(npa)
test_eq(data, xcp_calib.data[0].value)
# data

## convert a numpy array to a continuous hex string

In [ ]:
# npa.astype(np.float32).tobytes().hex()
buffer = npa.astype(np.float32).tobytes().hex()  ## == npa_to_packed_buffer(npa)
buffer, len(buffer)
test_eq(buffer, xcp_calib.data[0].value)

In [ ]:
#| export
def flash_xcp(xcp_calib: XCPCalib, data: pd.DataFrame, diff_flashing: bool=False, download: bool=True):
    """Summary
    Flash XCP data to target

    Args:
        xcp_calib (XCPCalib): XCP calibration as template, contains all the meta information except for data
        xcp_data (pd.DataFrame): input XCP data to be flashed, replace the value in xcp_calib
        diff_flashing (bool): Use differential flashing
        download (bool): Download or upload
    
    """
    
    # convert dataframe to a hex string to be flashed and assigned to XCPCalib field data
    xcp_calib.data = data.astype(np.float32).tobytes().hex()

    if download:
        if diff_flashing:
            raise NotImplementedError("Differential flashing not implemented yet")
        else:
            pass
        

    

In [ ]:
pkt = IP()
pkt.canvas_dump()

In [ ]:
IP()
a = IP(dst="10.10.10.28")
a.dst
a.ttl
ls(IP)

In [ ]:
os.getcwd()
a = rdpcap('../res/pcaps/ipfix.pcap')
a
# a[0].pdfdump(layer_shift=1)
a[1].psdump("/tmp/ipfix.eps", layer_shift=1)

# Caution

set the python3 of the virtualenv with the CAP_NET_RAW capability!

```bash
sudo setcap 'CAP_NET_RAW+eip CAP_NET_ADMIN+eip' /dpt/.pyenv/versions/miniconda3-3.11-24.1.2-0/envs/can/bin/python3.11
```

install sshpass, and create .sshpasswd.gpg file in home root
```bash
cd ~
sudo apt-get install sshpass
echo 'password_in_verbatim" > .sshpasswd
gpg -c .sshpasswd
ls | grep 'sshpasswd.gpg'
```

In [ ]:
# sniff(filter="icmp and host 10.10.10.28", count=2)

In [ ]:
args.channel_serial_number

In [ ]:
if blue_pill:
    # install vcan interface with encrypted password to sudo 
    os.system("gpg -d -q ~/.sshpasswd.gpg | sshpass -v sudo modprobe vcan")
    # sshpass -v -p asdf sudo ip link add dev vcan0 type vcan
    os.system(f"gpg -d -q ~/.sshpasswd.gpg | sshpass -v sudo ip link add dev vcan{args.channel_serial_number} type vcan")
    # !gpg -d -q ~/.sshpasswd.gpg | sshpass -v sudo ip link set vcan0 type vcan bitrate 500000  # vcan does not support set bitrate on command line!
    # !sshpass -p asdf sudo ip link add dev vcan0 type vcan
    os.system(f"gpg -d -q ~/.sshpasswd.gpg | sshpass -v sudo ip link set up vcan{args.channel_serial_number}")
    # !sshpass -v -p asdf sudo ip link set up vcan0
else:
    os.system(f"sudo modprobe vcan")
    os.system(f"sudo ip link add dev vcan{args.channel_serial_number} type vcan")
    os.system(f"sudo ip link set up vcan{args.channel_serial_number}")

os.system(f"ip link show vcan{args.channel_serial_number}")

In [ ]:
socket = CANSocket(channel=f'vcan{args.channel_serial_number}',
                receive_own_messages=True)

In [ ]:
packet = CAN(identifier=0x123, data=b'12345678')
packet.show2()

In [ ]:

socket.send(packet)
rx_packet = socket.recv()
rx_packet.show2()


In [ ]:
# rx_packet = socket.recv()
wrpcap("./scapypcaptest.pcap", packet)

In [ ]:
## another socket in the same process cannot receive the packet sent by the first socket
# socket2 = CANSocket(channel='vcan0')

In [ ]:
## same socket cannot receive the packet sent by itself
# rx_packet = socket2.recv()

# CCP via Scapy

## CRO for connection

In [ ]:
xcp_calib.config.download_can_id
hex(xcp_calib.config.download_can_id)

In [ ]:
channel = 'can' + str(xcp_calib.config.channel)
channel

In [ ]:
ctr = 0
print(f"ctr: {ctr}")
cro = CCP(identifier=xcp_calib.config.download_can_id)/CRO(ctr=ctr)/CONNECT()  # CONNECT(station_address=0x02)?
cro.show2()
cro.canvas_dump()


In [ ]:

socket.send(cro)
ctr += 1
rx_cro = socket.recv()
rx_cro.show2()
rx_cro.canvas_dump()
# dto = sock.sr1(cro)
# dto.show2()
# assert dto.return_code == 0x00

## Download from target

In [ ]:
xcp_calib.data

In [ ]:
d = xcp_calib.data[0]
d.address, d.type_size, d.dim, d.value_array_view, d.value_bytes
len(d.value_bytes )
d.value

## set mta

In [ ]:
print(f"ctr: {ctr}")
cro = CCP(identifier=xcp_calib.config.download_can_id)/CRO(ctr=ctr)/SET_MTA(address=int(d.address, 16))
cro.show2()
cro.payload
socket.send(cro)
ctr += 1
rx_cro = socket.recv()
rx_cro.show2()
# dto = sock.sr1(cro)
# dto.show2()
# assert dto.return_code == 0x00
rx_cro.canvas_dump()


In [ ]:
len_in_bytes = d.type_size * d.dim[0] * d.dim[1]
print(f"len_in_bytes: {len_in_bytes} = type_size: {d.type_size} x dim: {d.dim}")

download_times = len_in_bytes // 6
last_download_size = len_in_bytes % 6
print(f"download_times: {download_times}, last_download_size: {last_download_size}")


In [ ]:
d.value_bytes
len(d.value_bytes)
tile0 = d.value_bytes[0:6]
len(tile0), tile0, tile0.hex(), type(tile0)
tile1 = d.value_bytes[6:12]
len(tile1), tile1, tile1.hex()


In [ ]:
tile0 = d.value_bytes[0:4]
len(tile0), tile0, tile0.hex(), type(tile0), struct.unpack("<f", tile0)
tile1 = d.value_bytes[4:8]
len(tile1), tile1, tile1.hex(), struct.unpack("<f", tile1)
tile2 = d.value_bytes[8:12]
len(tile1), tile2, tile2.hex(), struct.unpack("<f", tile2)
st = 2*17*4
tile3 = d.value_bytes[st:st+4]
len(tile1), tile3, tile3.hex(), struct.unpack("<f", tile3)
d.value_array_view[2,0]
test_eq(struct.unpack("<f", tile3), d.value_array_view[2,0])

In [ ]:
tile3.hex(), tile3.hex().encode()

In [ ]:
type(tile3)
list(tile3)

In [ ]:
ba_uploaded = bytearray()
ba_uploaded += tile3
ba_uploaded
ba_uploaded += tile2
ba_uploaded += tile1
ba_uploaded, len(ba_uploaded)

In [ ]:
ba_uploaded = bytearray()
len_in_bytes = d.type_size * d.dim[0] * d.dim[1]
download_times = len_in_bytes // 6
last_download_size = len_in_bytes % 6
for tile in range(download_times):
    ba_uploaded += d.value_bytes[tile*6:(tile+1)*6]
if last_download_size:
    ba_uploaded += d.value_bytes[download_times*6:download_times*6+last_download_size]

len(ba_uploaded), ba_uploaded.hex()
test_eq(ba_uploaded, d.value_bytes)
test_eq(ba_uploaded.hex(), d.value)
ba_uploaded.hex()

## loop over XCPCalib data array

In [ ]:

for i in range(download_times):
    cro = CCP(identifier=xcp_calib.config.download_can_id)/CRO(ctr=ctr)/DNLOAD_6(data=d.value_bytes[i*6:(i+1)*6])
    if i%100==0:
        print(f"i: {i},  ctr: {ctr}, cro: {cro}")
        cro.show2()
        cro.payload
    sent_bytes = socket.send(cro)
    ctr += 1
    rx_cro = socket.recv()
    # rx_cro.show2()
    # dto = sock.sr1(cro)
    # dto.show2()
    # assert dto.return_code == 0x00
i
cro.canvas_dump()
rx_cro.canvas_dump()

In [ ]:
start_index = download_times * 6
cro = CCP(identifier=xcp_calib.config.download_can_id)/CRO(ctr=ctr)/DNLOAD(data=d.value_bytes[start_index:start_index+last_download_size])
print(f"ctr: {ctr}")
cro.show2()
cro.payload
cro.canvas_dump()
sent_bytes = socket.send(cro)
ctr += 1 
rx_cro = socket.recv()
rx_cro.canvas_dump()

## Disconnect target ecu

In [ ]:

cro = CCP(identifier=xcp_calib.config.download_can_id)/CRO(ctr=ctr)/DISCONNECT(station_address=0x00)
print(f"ctr: {ctr}")
cro.show2()
cro.canvas_dump()
bytes_sent = socket.send(cro)
ctr += 1
cx_cro = socket.recv()
cx_cro.show2()
cx_cro.canvas_dump()


## download_calib_data

In [ ]:
#| export
def downlod_calib_data(xcp_calib: XCPCalib, 
                        can_type: CANType, 
                        channel: int,
                        bus_type: BusType, 
                        can_filter=list[dict],  
                        bit_rate: int=500_000, 
                        timeout: float=1.0,
                        diff_flashing: bool=False):
    """Summary
    Download XCP calibration data to target

    Args:
        xcp_calib (XCPCalib): XCP calibration  to be downloaded into the target
        diff_flashing (bool): Use differential flashing
    """
    # init counter
    ctr = 0
    # create a socket
    match can_type:
        case 'NATIVE':
            load_layer("can")
            conf.contribs['CANSocket'] = {'use-python-can': False}
            load_contrib("cansocket")
            match bus_type:
                case 'SOCKET':
                    sock = CANSocket(channel='can'+str(channel), can_filter=can_filter, bit_rate = bit_rate, receive_own_messages=True)
                case 'VIRTUAL':
                    sock = CANSocket(channel='vcan'+str(channel), can_filter=can_filter, bit_rate = bit_rate, receive_own_messages=True)
                case _:
                    raise ValueError(f"Invalid CAN bus type: {bus_type}, valid types are: SOCKET or VIRTUAL for Native CANSOCKET")
        case 'PYTHON': 
            assert bus_type is not None, "Bus type must be specified for PYTHON CAN"
            load_layer("can")
            conf.contribs['CANSocket'] = {'use-python-can': True}
            load_contrib("cansocket")
            match bus_type:
                case 'SOCKET':
                    sock = CANSocket(bustype='socketcan', channel='can'+channel, can_filter=can_filter, bitrate=bit_rate, receive_own_messages=True)
                case 'VIRTUAL':
                    sock = CANSocket(bustype='socketcan', channel='vcan'+channel, can_filter=can_filter, bitrate=bit_rate, receive_own_messages=True)
                case 'KVASER':
                    sock = CANSocket(bustype='kvaser', channel=channel, can_filter=can_filter, bitrate=bit_rate, receive_own_messages=True)
                case 'VECTOR':
                    sock = CANSocket(bustype='vector', channel=channel, can_filter=can_filter, bitrate=bit_rate, receive_own_messages=True)
                case _:
                    raise ValueError(f"Invalid CAN bus type: {bus_type}, implemented valid types are: SOCKET, KVASER, VECTOR for Python-CAN CANSOCKET")

    # CONNECT
    ctr += 1
    cro = CCP(identifier=xcp_calib.config.download_can_id)/CRO(ctr=ctr)/CONNECT(station_address=0x00)
    dto = sock.sr1(cro, timeout=timeout)
    assert dto.return_code == 0x00

    for d in xcp_calib.data:
        # SET_MTA
        ctr += 1
        cro = CCP(identifier=xcp_calib.config.download_can_id)/CRO(ctr=ctr)/SET_MTA(address=int(d.address, 16))
        dto = sock.sr1(cro, timeout=timeout)
        assert dto.return_code == 0x00

        # Determine message tiling
        len_in_bytes = d.type_size * d.dim[0] * d.dim[1]
        assert len_in_bytes == len(d.value_bytes)
        tile_size = 6  # 6 bytes per tile as defined in CCP for DNLOAD_6
        tiles = len_in_bytes // tile_size 
        last_tile = len_in_bytes % tile_size
        # Download full size tiles with DNLOAD_6
        for i in range(tiles):
            ctr += 1
            cro = CCP(identifier=xcp_calib.config.download_can_id)/CRO(ctr=ctr)/DNLOAD_6(data=d.value_bytes[i*tile_size:(i+1)*tile_size])
            dto = sock.sr1(cro,timeout=timeout)
            assert dto.return_code == 0x00
        start_index = tiles * tile_size
        ctr += 1 
        cro = CCP(identifier=xcp_calib.config.download_can_id)/CRO(ctr=ctr)/DNLOAD(data=d.value_bytes[start_index:start_index+last_tile])
        dto = sock.sr1(cro,timeout=timeout)
        assert dto.return_code == 0x00

    # DISCONNECT
    ctr += 1
    cro = CCP(identifier=xcp_calib.config.download_can_id)/CRO(ctr=ctr)/DISCONNECT(station_address=0x00)
    dto = sock.sr1(cro,timeout=timeout)
    assert dto.return_code == 0x00

## upload_calib_data

In [ ]:
#| export
def upload_calib_data(xcp_calib: XCPCalib, 
                        can_type: CANType, 
                        channel: int,
                        bus_type: BusType, 
                        can_filter=list[dict],  
                        bit_rate: int=500_000, 
                        timeout: float=1.0,
                        diff_flashing: bool=False)->None:
    """Summary
    Upload XCP calibration data from target to host, the result will update the xcp_calib.data field

    Args:
        xcp_calib (XCPCalib): XCP calibration  to be uploaded from the target to host
        diff_flashing (bool): Use differential flashing
    """

    # init counter
    ctr = 0
    # create a socket
    match can_type:
        case 'NATIVE':
            load_layer("can")
            conf.contribs['CANSocket'] = {'use-python-can': False}
            load_contrib("cansocket")
            match bus_type:
                case 'SOCKET':
                    sock = CANSocket(channel='can'+str(channel), can_filter=can_filter, bit_rate = bit_rate, receive_own_messages=True)
                case 'VIRTUAL':
                    sock = CANSocket(channel='vcan'+str(channel), can_filter=can_filter, bit_rate = bit_rate, receive_own_messages=True)
                case _:
                    raise ValueError(f"Invalid CAN bus type: {bus_type}, valid types are: SOCKET or VIRTUAL for Native CANSOCKET")
        case 'PYTHON': 
            assert bus_type is not None, "Bus type must be specified for PYTHON CAN"
            load_layer("can")
            conf.contribs['CANSocket'] = {'use-python-can': True}
            load_contrib("cansocket")
            match bus_type:
                case 'SOCKET':
                    sock = CANSocket(bustype='socketcan', channel='can'+channel, can_filter=can_filter, bitrate=bit_rate, receive_own_messages=True)
                case 'VIRTUAL':
                    sock = CANSocket(bustype='socketcan', channel='vcan'+channel, can_filter=can_filter, bitrate=bit_rate, receive_own_messages=True)
                case 'KVASER':
                    sock = CANSocket(bustype='kvaser', channel=channel, can_filter=can_filter, bitrate=bit_rate, receive_own_messages=True)
                case 'VECTOR':
                    sock = CANSocket(bustype='vector', channel=channel, can_filter=can_filter, bitrate=bit_rate, receive_own_messages=True)
                case _:
                    raise ValueError(f"Invalid CAN bus type: {bus_type}, implemented valid types are: SOCKET, KVASER, VECTOR for Python-CAN CANSOCKET")
    # sock = CANSocket(channel='can'+str(xcp_calib.config.channel), receive_own_messages=True)

    # CONNECT
    ctr += 1
    cro = CCP(identifier=xcp_calib.config.download_can_id)/CRO(ctr=ctr)/CONNECT(station_address=0x00)
    dto = sock.sr1(cro,timeout=timeout)
    assert dto.return_code == 0x00

    for d in xcp_calib.data:
        # SET_MTA
        ctr += 1
        cro = CCP(identifier=xcp_calib.config.download_can_id)/CRO(ctr=ctr)/SET_MTA(address=int(d.address, 16))
        dto = sock.sr1(cro,timeout=timeout)
        assert dto.return_code == 0x00

        # Determine message tiling
        len_in_bytes = d.type_size * d.dim[0] * d.dim[1]
        # assert len_in_bytes == len(d.value_bytes)
        tile_size = 5
        tiles = len_in_bytes // tile_size 
        last_tile = len_in_bytes % tile_size
        
        # Upload tiles with tile_size (maximal 5 as defined by CCP） bytes with UPLOAD
        ba_uploaded = bytearray()
        for i in range(tiles):
            ctr += 1
            cro = CCP(identifier=xcp_calib.config.download_can_id)/CRO(ctr=ctr)/UPLOAD(size=tile_size)
            upload_dto = sock.sr1(cro,timeout=timeout)
            assert upload_dto.return_code == 0x00
            ba_uploaded += upload_dto.data
            

        start_index = tiles * tile_size
        ctr += 1 
        cro = CCP(identifier=xcp_calib.config.download_can_id)/CRO(ctr=ctr)/UPLOAD(last_tile)
        upload_dto = sock.sr1(cro,timeout=timeout)
        assert upload_dto.return_code == 0x00
        ba_uploaded += upload_dto.data

        d.value = ba_uploaded.hex()

    # DISCONNECT
    ctr += 1
    cro = CCP(identifier=xcp_calib.config.download_can_id)/CRO(ctr=ctr)/DISCONNECT(station_address=0x00)
    dto = sock.sr1(cro,timeout=timeout)
    assert dto.return_code == 0x00

## Three context managers for CCP

In [ ]:
#| export
@contextlib.contextmanager
def can_context(can_specs: ScapyCANSpecs):
    """Summary
    Context manager for scapy CAN socket

    Args:
        can_specs (ScapyCANSpecs): CAN specs including can type, bus type, channel, etc.

    Yields:
        CANSocket: CAN socket object
    """
    # create a socket
    try:
        match can_specs.can_type:
            case 'NATIVE':
                if not 'can' in sys.modules:
                    load_layer("can")
                if not 'cansocket' in conf.contribs:
                    conf.contribs['CANSocket'] = {'use-python-can': False}
                    load_contrib("cansocket")
                sock = CANSocket(channel=can_specs.channel, 
                                    can_filter=can_specs.can_filters, 
                                    bit_rate = can_specs.bit_rate, 
                                    receive_own_messages=can_specs.receive_own_messages
                                )
            case 'PYTHON': 
                if not 'can' in sys.modules:
                    load_layer("can")
                if not 'cansocket' in conf.contribs:
                    conf.contribs['CANSocket'] = {'use-python-can': True}
                    load_contrib("cansocket")
                match bus_type:
                    case 'SOCKET' | 'VIRTUAL':
                        sock = CANSocket(bustype='socketcan', 
                                            channel=can_specs.channel, 
                                            can_filter=can_specs.can_filters, 
                                            bitrate=can_specs.bit_rate, 
                                            receive_own_messages=can_specs.receive_own_messages
                                        )
                    case 'KVASER' | 'VECTOR':
                        sock = CANSocket(bustype=can_specs.bus_type.lower(), 
                                            channel=can_specs.channel, 
                                            can_filter=can_specs.can_filters, 
                                            bitrate=can_specs.bit_rate, 
                                            receive_own_messages=can_specs.receive_own_messages)
                    case _:
                        raise ValueError(f"Invalid CAN bus type: {can_specs.bus_type}, implemented valid types are: SOCKET, VIRTUAL, KVASER, VECTOR")
    except Exception as e:
        raise Exception(f"Failed to create CAN socket: {e}")

    # CONNECT
    cro = CCP(identifier=can_specs.download_can_id)/CRO(ctr=can_specs.cntr)/CONNECT(station_address=can_specs.station_address)
    dto = sock.sr1(cro, timeout=can_specs.time_out)
    can_specs.cntr += 1
    assert dto is not None, f"Failed to connect to target, timeout={can_specs.time_out} seconds"
    assert dto.return_code == 0x00
    
    try: 
        yield sock
    except TimeoutError:
        raise TimeoutError(f"CAN socket timeout: {can_specs.time_out} seconds")
    except Exception as e:
        raise e
    finally:
        # DISCONNECT
        cro = CCP(identifier=can_specs.download_can_id)/CRO(ctr=can_specs.cntr)/DISCONNECT(station_address=can_specs.station_address)
        dto = sock.sr1(cro, timeout=can_specs.time_out)
        can_specs.cntr += 1
        assert dto.return_code == 0x00

In [ ]:
#| export
@contextlib.contextmanager
def SET_MTA_context(can_specs: ScapyCANSpecs, sock: CANSocket, data: XCPData) -> CAN:
    """Summary
    Context manager for scapy set_mta 

    Args:
        channel (str): CAN channel to use, default is vcan0

    Yields:
        CAN: packdet for CAN message 
    """

    # SET_MTA 
    cro = CCP(identifier=can_specs.download_can_id)/CRO(ctr=can_specs.cntr)/SET_MTA(address=int(data.address, 16))
    dto = sock.sr1(cro, timeout=can_specs.time_out)
    can_specs.cntr += 1
    assert dto.return_code == 0x00
    try:
        yield dto
    except TimeoutError:
        raise TimeoutError(f"CAN socket timeout: {can_specs.time_out} seconds")
    except Exception as e:
        raise e
    finally:
        pass  # do nothing, just pray it'll be OK. Crapy CCP!
    


In [ ]:
#| export
@contextlib.contextmanager
def XLOAD_context(can_specs: ScapyCANSpecs, sock: CANSocket, data: XCPData, start_index: int, tile_size: int):
    """Summary
    Context manager for scapy load (download or upload)

    Args:
        channel (str): CAN channel to use, default is vcan0

    Yields:
        CANSocket: CAN socket object
    """

    if can_specs.download_upload:
        assert tile_size <=6 and tile_size >0, f"In CCP Tile size must be non-zero and less than 6 bytes, got: {tile_size}"
        if tile_size == 6:
            # cro = CCP(identifier=xcp_calib.config.download_can_id)/CRO(ctr=ctr)/DNLOAD_6(data=d.value_bytes[i*tile_size:(i+1)*tile_size])
            cro = CCP(identifier=can_specs.download_can_id)/CRO(ctr=can_specs.cntr)/DNLOAD_6(data=d.value_bytes[start_index:start_index+tile_size])
            dto = sock.sr1(cro,timeout=can_specs.time_out)
            assert dto.return_code == 0x00
        else: 
            # start_index = tiles * tile_size
            # cro = CCP(identifier=xcp_calib.config.download_can_id)/CRO(ctr=ctr)/DNLOAD(data=d.value_bytes[start_index:start_index+last_tile])
            cro = CCP(identifier=can_specs.download_can_id)/CRO(ctr=can_specs.cntr)/DNLOAD(data=d.value_bytes[start_index:start_index+tile_size])
            dto = sock.sr1(cro,timeout=can_specs.time_out)
            assert dto.return_code == 0x00
    else:  # False if in upload mode
        assert tile_size <=5 and tile_size >0, f"In CCP Tile size must be non-zero and less than 5 bytes for Uploading, got: {tile_size}"
        # cro = CCP(identifier=xcp_calib.config.download_can_id)/CRO(ctr=ctr)/DNLOAD_6(data=d.value_bytes[i*tile_size:(i+1)*tile_size])
        cro = CCP(identifier=can_specs.download_can_id)/CRO(ctr=can_specs.cntr)/UPLOAD(size=tile_size)  #start_index or data not used for uploading. target ECU will increment the address!
        dto = sock.sr1(cro,timeout=can_specs.time_out)
        assert dto.return_code == 0x00
    
    can_specs.cntr += 1
    try: 
        yield dto
    except TimeoutError:
        raise TimeoutError(f"CAN socket timeout: {can_specs.time_out} seconds")
    except Exception as e:
        raise e
    finally:
        pass  # do nothing, just pray it'll be OK. Crapy CCP!

In [ ]:
can_filters = [{'can_id': xcp_calib.config.upload_can_id, 'can_mask': 0x7FF}]
can_specs = ScapyCANSpecs(can_type='NATIVE', 
                        bus_type='VIRTUAL', 
                        channel_serial_number=0,
                        download_can_id=xcp_calib.config.download_can_id,
                        upload_can_id=xcp_calib.config.upload_can_id,
                        can_filters=can_filters,
                        bit_rate=500_000,
                        time_out=1.0,
                        station_address=0x00,
                        cntr=0,
                        receive_own_messages=True,
                        download_upload=True
                        )
can_specs
can_specs.model_dump()

## Downloading and uploading with context managers

In [ ]:
#| export
def upload_calib_data2(xcp_calib: XCPCalib, 
                        can_specs: ScapyCANSpecs
                        )->None:

    """Summary
    Upload XCP calibration data from target to host, the result will update the xcp_calib.data field

    Args:
        xcp_calib (XCPCalib): XCP calibration  to be uploaded from the target to host
        diff_flashing (bool): Use differential flashing
    """

    # init counter
    cntr = 0

    try:
        with can_context(can_specs=can_specs) as sock:
            for d in xcp_calib.data:
                # SET_MTA
                with SET_MTA_context(can_specs=can_specs, sock=sock, data=d) as dto:
                    assert dto.return_code==0x00, f"SET_MTA failed for {d.name} at {d.address}"

                    # Determine message tiling
                    len_in_bytes = d.type_size * d.dim[0] * d.dim[1]
                    # assert len_in_bytes == len(d.value_bytes)
                    tile_size = 5
                    tiles = len_in_bytes // tile_size 
                    last_tile = len_in_bytes % tile_size

                    # Upload tiles with tile_size (maximal 5 as defined by CCP） bytes with UPLOAD
                    ba_uploaded = bytearray()
                    for i in range(tiles):
                        with XLOAD_context(can_specs=can_specs, sock=sock, data=d, start_index=i*tile_size, tile_size=tile_size) as upload_dto:
                            assert upload_dto.return_code == 0x00, f"UPLOAD failed at tile: {i}"
                            ba_uploaded += upload_dto.data

                    with XLOAD_context(can_specs=can_specs, sock=sock, data=d, start_index=i*tile_size, tile_size=last_tile) as upload_dto:
                        assert upload_dto.return_code == 0x00, f"UPLOAD failed at tile: {i}"
                        ba_uploaded += upload_dto.data
                        ba_uploaded += upload_dto.data

                    d.value = ba_uploaded.hex()
    except Exception as e:
        print(e)

In [ ]:
#| export
def downlod_calib_data2(xcp_calib: XCPCalib, 
                        can_specs: ScapyCANSpecs
                        )->None:
    """Summary
    Download XCP calibration data to target use scapy_can_context

    Args:
        xcp_calib (XCPCalib): XCP calibration  to be downloaded into the target
    """
    # init counter
    cntr = 0

    if can_specs.diff_mode :
        if can_specs.last_download_data is None:  # diff mode in the first run, needs to upload first to populate last_download_data
            last_xcp_calib = XCPCalib(config=xcp_calib.config, data=xcp_calib.data)
            can_specs.download_upload = False
            upload_calib_data2(xcp_calib=last_xcp_calib, can_specs=can_specs)
            can_specs.last_download_data = last_xcp_calib.data
            can_specs.download_upload = True
        # calculate the difference between the last downloaded data and the current data
        assert len(can_specs.last_download_data)==len(xcp_calib.data), "XCPData list length is not the same"
        # data_pair = zip(can_specs.last_download_data, xcp_calib.data)
        xcp_data = []
        for d0, d1 in zip(xcp_calib.data, can_specs.last_download_data):
            assert d0.is_compatible(d1), f"incompatible data {d0} vs {d1}"
            diff_array_index_2d = np.where((d0.value_array_view - d1.value_array_view) > can_specs.diff_threshold)
            diff_array_index_1d = np.ravel_multi_index(diff_array_index_2d, d0.dim, order='C')
            diff_array_value = d0[diff_array_index_2d]
            diff_array_address = d0.address_int + diff_array_index_1d * d0.type_size
            
            xcp_data += [XCPData(address=hex(address), 
                                value=value, 
                                name=d0.name, 
                                dim=d0.dim, 
                                value_type=d0.value_type, 
                                value_length=d0.value_length
                                ) 
                            for address, value in zip(diff_array_address, diff_array_value)
                        ]
    else:  # non-diff mode 
        xcp_data = xcp_calib.data
        
    try:
        with can_context(can_specs=can_specs) as sock:
            for d in xcp_data:
                # SET_MTA
                with SET_MTA_context(can_specs=can_specs, sock=sock, data=d) as dto:
                    assert dto.return_code==0x00, f"SET_MTA failed for {d.name} at {d.address}"
                    # Determine message tiling
                    len_in_bytes = d.type_size * d.dim[0] * d.dim[1]
                    assert len_in_bytes == len(d.value_bytes)
                    tile_size = 6  # 6 bytes per tile as defined in CCP for DNLOAD_6
                    tiles = len_in_bytes //tile_size 
                    last_tile = len_in_bytes % tile_size
                    # Download full size tiles with DNLOAD_6
                    for i in range(tiles):
                        start_index = i*tile_size
                        with XLOAD_context(can_specs=can_specs, sock=sock, data=d, start_index=start_index, tile_size=tile_size) as dto:
                            assert dto.return_code == 0x00, f"DNLOAD_6 failed at tile: {i}"

                    start_index = tiles * tile_size
                    with XLOAD_context(can_specs=can_specs, sock=sock, data=d, start_index=start_index, tile_size=last_tile) as dto:
                        assert dto.return_code == 0x00, f"DNLOAD failed at last tile: {i} of size {last_tile}"
    except Exception as e:
        print(e)
    
    # keep the last downloaded data for diff mode
    can_specs.last_download_data = xcp_calib.data

In [ ]:
xcp_calib.config.channel = 0
xcp_calib

In [ ]:
# TODO cannot run test on CCP without a real or emulated ECU 
# downlod_calib_data2(xcp_calib, can_type='NATIVE', bus_type='VIRTUAL', bit_rate=500_000, timeout=1.0, station_address=0x00, diff_flashing=False)

# Release CAN device

In [ ]:
if blue_pill:
    # close and remove vcan0
    # !sshpass -v -p  asdf sudo ip link delete vcan0 
    os.system(f"gpg -d -q ~/.sshpasswd.gpg | sshpass -v sudo ip link set down vcan{args.channel_serial_number}")
    # delete vcan0
    os.system(f"gpg -d -q ~/.sshpasswd.gpg | sshpass -v sudo ip link delete vcan{args.channel_serial_number}")
else:
    os.system(f"sudo ip link set down vcan{args.channel_serial_number}")
    os.system(f"sudo ip link delete vcan{args.channel_serial_number}")

os.system(f"ip link show vcan{args.channel_serial_number}")

In [ ]:
#| export
if __name__ == "__main__" and "__file__" in globals():  # only run if this file is called directly

    protocol = inquirer.select(
        message="What's the protocol?",
        choices=[
            Choice(value="CCP", name="CCP"),
            Choice(value="XCP", name="XCP"),
        ],
        default="CCP",
    ).execute()

    can = inquirer.select(
        message="What's the type of CAN?",
        choices=[
            Choice(value="NATIVE", name="Native Linux SocketCAN"),
            Choice(value="PYTHON", name="Python CAN"),
        ],
        default="NATIVE",
    ).execute()

    if can == 'NATIVE':
        bus = inquirer.select(
            message="What's the type of bus?",
            choices=[
                Choice(value="SOCKET", name="Physical CAN"),
                Choice(value="VIRTUAL", name="Virtual CAN"),
            ],
            default="SOCKET",
        ).execute()
    else:  # can == 'PYTHON'
        bus = inquirer.select(
            message="What's the type of bus?",
            choices=[
                Choice(value="SOCKET", name="Physical SocketCAN"),
                Choice(value="VIRTUAL", name="Virtual SocketCAN"),
                Choice(value="KVASER", name="Kvaser CAN"),
            ],
            default="SOCKET",
        ).execute()

    download = inquirer.confirm(
        message="Downloading(host->target)?",
        confirm_letter="y",
        reject_letter="n",
        default=True,
    ).execute()

    differential_flashing = inquirer.confirm(
        message="Differential Flashing?",
        confirm_letter="y",
        reject_letter="n",
        default=True,
    ).execute()

    a2l_file_path = inquirer.text(
        message="a2l file path",
        validate=EmptyInputValidator(),
        default='/res/vbu_ai.json'
    ).execute()

    # node_path = inquirer.text(
    #     message="node path",
    #     validate=EmptyInputValidator(),
    # 	default=r"/PROJECT/MODULE[]",
    # ).execute()

    # leaves = inquirer.text(
    #     message="leaves",
    #     validate=EmptyInputValidator(),
    # 	default=r"TQD_trqTrqSetNormal_MAP_v, VBU_L045A_CWP_05_09T_AImode_CM_single, Lookup2D_FLOAT32_IEEE, Lookup2D_X_FLOAT32_IEEE, Scalar_FLOAT32_IEEE, TQD_vVehSpd, TQD_vSgndSpd_MAP_y, TQD_pctAccPedPosFlt, TQD_pctAccPdl_MAP_x"
    # ).execute()

    # can_channel = inquirer.number(
    #     message="CAN channel for flashing",
    #     min_allowed=0,
    #     max_allowed=32,
    #     validate=EmptyInputValidator(),
    #     default=3,
    # ).execute()

    input_file_path = inquirer.text(
        message="Input file path",
        validate=EmptyInputValidator(),
        default="/res/download.json",
    ).execute()

    output_file_path = inquirer.text(
        message="Output file path",
        validate=EmptyInputValidator(),
        default="/res/output.json",
    ).execute()

    args = get_argparser().parse_args()
    args.protocol = protocol
    args.can_type = can
    args.bus_type = bus
    args.download = download
    args.diff_mode = differential_flashing
    # args.a2l = a2l_file_path
    # args.node_path = node_path
    # args.leaves = leaves
    args.input = repo.working_dir+input_file_path
    args.output = repo.working_dir+output_file_path
    pprint(args)

    xcp_calib_from_xcpjson = Get_XCPCalib_From_XCPJSon(args.input)

    args.download_can_id = xcp_calib_from_xcpjson.config.download_can_id
    args.upload_can_id = xcp_calib_from_xcpjson.config.upload_can_id
    args.channel_serial_number = xcp_calib_from_xcpjson.config.channel

    xcp_data = Generate_Init_XCPData_From_A2L(
        a2l=args.a2l, keys=args.leaves, node_path=args.node_path
    )  # initial xcp_data has value 0
    try:
        XCPData.model_validate(xcp_data)
    except ValidationError as exc:
        print(exc)

    # emulate torque table input as numpy array
    xcp_data_value_npa = xcp_calib_from_xcpjson.data[0].value_array_view
    xcp_data.value = xcp_data_value_npa.astype(np.float32).tobytes().hex()
    pprint(xcp_data)

    xcp_calib = XCPCalib(
        config=XCPConfig(
            channel=args.channel_serial_number, download=args.download_can_id, upload=args.upload_can_id
        ),
        data=[xcp_data],
    )
    pprint(xcp_calib)

    can_filters = [{'can_id': xcp_calib.config.upload_can_id, 'can_mask': 0x7FF}]
    cntr = 0
    can_specs = ScapyCANSpecs(can_type=args.can_type,
                            bus_type=args.bus_type,
                            channel_serial_number=args.channel_serial_number,
                            download_can_id=xcp_calib.config.download_can_id,
                            upload_can_id=xcp_calib.config.upload_can_id,
                            can_filters=can_filters,
                            bit_rate=args.bit_rate,
                            time_out=args.time_out,
                            station_address=args.station_address,
                            cntr=cntr,
                            receive_own_messages=True,
                            download_upload=args.download,  # CCP Upload mode
                            diff_mode = args.diff_mode,
                            diff_threshold= args.diff_threshold
                            )
    can_specs.download_upload = False                        
    upload_calib_data2(xcp_calib=xcp_calib, can_specs=can_specs)
    can_specs.download_upload = True                       
    downlod_calib_data2(xcp_calib=xcp_calib,can_specs=can_specs)
    

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()